In [1]:
# Cloud Mask 2.0 

import ee
import yaml
import time
import os
import json
from argparse import ArgumentParser
from utils import clipToROI, exportImageCollectionToGCS, exportImageToGCS, sentinel2CloudScore, calcCloudCoverage
from utils import GEETaskManager

from gevent.fileobject import FileObjectThread

# Polygon Import from Misha ROI List

import pandas as pd

# Functions for Active Run of Cloud Mask 2.0 

from download_sen12 import *

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
config_file = "config.yml"

In [4]:
stream = open(config_file, 'r') 

In [5]:
config = yaml.load(stream)

<ipython-input-5-8d98dc56653f>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(stream)


In [8]:
config["data_list"][0]

{'name': ['summer', '1886'],
 'start_date': '2017-06-01',
 'end_date': '2017-08-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'name',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

# Initialize EE

In [9]:
ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


# Creating New Feature Collection To Use with CM_v2

In [20]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

polygons = import_aois("D:/canopy_data/csvs/polygons_101320.csv")

feature_id = 0 
features = []
for poly in polygons[0:3]:
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)

fc = ee.FeatureCollection(features)

In [11]:
fc

In [12]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list'][0:1]:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        print(sensor)
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'], feature_list = fc)

print("Waiting for completion...")
task_queue.wait_till_done()

{'name': 'COPERNICUS/S2', 'prefix': 'S2_CloudFree', 'type': 'opt', 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]}
3 features have been loaded
Queued Task S2_CloudFree_summer_1886_1
Processing S2_CloudFree_summer_1886_1
Task [S2_CloudFree_summer_1886_1] has failed to complete
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
Queued Task S2_CloudFree_summer_1886_2
{'name': 'COPERNICUS/S2', 'type': 'opt', 'prefix': 'S2_20_60_Cloud', 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'gte': 20}}, {'CLOUDY_PERCENTAGE': {'lte': 60}}]}
Processing S2_CloudFree_summer_1886_2
Task [S2_CloudFree_summer_1886_2] has failed to complete
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers
3 features have been loaded
Queued Task S2_20_60_Cloud_summer_1886

AttributeError: 'NoneType' object has no attribute 'rawlink'

SAVING LOG
Processing S1_summer_1886_2
SAVING LOG
Task [S1_summer_1886_2] has failed to complete
Worker 0 has no more work... Returning
Monitor has quit as there are no more workers


In [1]:
d = {'a': 1, 'b': 2, 'c': 3}

d.keys()

dict_keys(['a', 'b', 'c'])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
list(d.keys())[0]

In [ ]:
list(d.values())[0]

In [3]:
d.items()

dict_items([('a', 1), ('b', 2), ('c', 3)])

In [ ]:
e = {'d': d, 'e': 0}

e

In [ ]:
list(list(e.values())[0].keys())[0]

In [ ]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list']:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'])

print("Waiting for completion...")
task_queue.wait_till_done()

In [ ]:
!pwd

## makeFilterList

In [2]:
def makeFilterList(sensor):
    filters_before = None
    filters_after = None

    def _build_filters(filter_list):
        filters = []
        for f in filter_list:
            key = list(f.keys())[0]
            op = list(list(f.values())[0].keys())[0]
            val = list(list(f.values())[0].values())[0]
            filters.append(getattr(ee.Filter, op)(key, val))

        return filters

    if 'filters_before' in sensor:
        filters_before = _build_filters(sensor['filters_before'])

    if 'filters_after' in sensor:
        filters_after = _build_filters(sensor['filters_after'])

    return filters_before, filters_after

In [6]:
type(config)

dict

In [7]:
config.keys()

dict_keys(['downloadDirectory', 'export_to', 'export_dest', 'log_file', 'max_tasks', 'max_retry', 'task_timeout', 'sensors', 'data_list'])

In [10]:
config

{'downloadDirectory': '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/test_cloud_mask_v2/DL_test',
 'export_to': 'bucket',
 'export_dest': 'sentinel2',
 'log_file': 'gee_download_log.pkl',
 'max_tasks': 5,
 'max_retry': 1,
 'task_timeout': 14400,
 'sensors': [{'name': 'COPERNICUS/S2',
   'prefix': 'S2_CloudFree',
   'type': 'opt',
   'bands': ['B1',
    'B2',
    'B3',
    'B4',
    'B5',
    'B6',
    'B7',
    'B8',
    'B8A',
    'B9',
    'B10',
    'B11',
    'B12'],
   'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]},
  {'name': 'COPERNICUS/S2',
   'type': 'opt',
   'prefix': 'S2_20_60_Cloud',
   'bands': ['B1',
    'B2',
    'B3',
    'B4',
    'B5',
    'B6',
    'B7',
    'B8',
    'B8A',
    'B9',
    'B10',
    'B11',
    'B12'],
   'filters_after': [{'CLOUDY_PERCENTAGE': {'gte': 20}},
    {'CLOUDY_PERCENTAGE': {'lte': 60}}]},
  {'name': 'COPERNICUS/S1_GRD',
   'type': 'sar',
   'prefix': 'S1',
   'bands': ['VV', 'VH'],
   'filters_before': [{'

In [8]:
config['sensors']

[{'name': 'COPERNICUS/S2',
  'prefix': 'S2_CloudFree',
  'type': 'opt',
  'bands': ['B1',
   'B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B8',
   'B8A',
   'B9',
   'B10',
   'B11',
   'B12'],
  'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]},
 {'name': 'COPERNICUS/S2',
  'type': 'opt',
  'prefix': 'S2_20_60_Cloud',
  'bands': ['B1',
   'B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B8',
   'B8A',
   'B9',
   'B10',
   'B11',
   'B12'],
  'filters_after': [{'CLOUDY_PERCENTAGE': {'gte': 20}},
   {'CLOUDY_PERCENTAGE': {'lte': 60}}]},
 {'name': 'COPERNICUS/S1_GRD',
  'type': 'sar',
  'prefix': 'S1',
  'bands': ['VV', 'VH'],
  'filters_before': [{'instrumentMode': {'eq': 'IW'}},
   {'transmitterReceiverPolarisation': {'listContains': 'VV'}},
   {'transmitterReceiverPolarisation': {'listContains': 'VH'}}]}]

In [9]:
sensor = config['sensors'][0]
sensor

{'name': 'COPERNICUS/S2',
 'prefix': 'S2_CloudFree',
 'type': 'opt',
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B10',
  'B11',
  'B12'],
 'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]}

In [11]:
# Default values
filters_before = None
filters_after = None

In [ ]:
# Sub function
def _build_filters(filter_list):
    # filter_list is a list of dictionaries. Includes the attributes for filtering an image collection
    filters = []
    # for each dict in filter_list
    # example: {'CLOUDY_PERCENTAGE': {'lte': 10}}
    for f in filter_list:
        # key is the first key of the dict -- the feature you're trying to filter by
        # example: 'CLOUDY_PERCENTAGE'
        key = list(f.keys())[0]
        # op is the key of the nested dictionary
        # example: 'lte'
        op = list(list(f.values())[0].keys())[0]
        # val is the value of the nested dictionary
        # example: 10
        val = list(list(f.values())[0].values())[0]
        # Make an ee.Filter object that matches the input filter dict
        # example: ee.Filter.lte('CLOUDY_PERCENTAGE', 10)
        # This will then get applied to an image_collection object
        filters.append(getattr(ee.Filter, op)(key, val))

In [12]:
help(ee.Filter)

Help on class Filter in module ee.filter:

class Filter(ee.computedobject.ComputedObject)
 |  Filter(*args, **kwargs)
 |  
 |  An object to represent collection filters.
 |  
 |  Method resolution order:
 |      Filter
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Not(self)
 |      Returns the opposite of this filter.
 |      
 |      Returns:
 |        The negated filter, which will match iff this filter doesn't.
 |  
 |  __init__(self, opt_filter=None)
 |      Construct a filter.
 |      
 |      This constructor accepts the following args:
 |        1) Another filter.
 |        2) An array of filters (which are implicitly ANDed together).
 |        3) A ComputedObject returning a filter. Users shouldn't be making these;
 |           they're produced by the generator functions below.
 |      
 |      Args:
 |        opt_filter: Optional filter to add.
 |  
 |  predicateCount(self)
 |      Return

In [ ]:
# Because of the sorting (probably), you may want to apply filters specifically
# before or after creating the image_collection object.
# So we have separate filter lists for both before and after.
if 'filters_before' in sensor:
    filters_before = _build_filters(sensor['filters_before'])

if 'filters_after' in sensor:
    filters_after = _build_filters(sensor['filters_after'])
    
# So at the end, we build a list of ee.Filter objects based off of the sensor
# values for its 'filters_before' and 'filters_after' keys. If the sensor
# lacks one or both such keys, the filters_before and filters_after retain
# their default None value (i.e. no filters get applied).

In [14]:
ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [15]:
makeFilterList(sensor)

(None, [<ee.filter.Filter at 0x25c4c5754c0>])

## makeImageCollection

In [17]:
def makeImageCollection(sensor, roi, start_date, end_date, modifiers=[]):
    # Make the filters based off of the previous function
    filters_before, filters_after = makeFilterList(sensor)

    # Make an image collection. Take the name from the sensor.
    # Filter by date based off of start_date and end_date.
    # Filter bounds based off of the ROI.
    # The map method applies an additional function as a filter; in this case,
    # a clipToROI function that crops every image result in the collection.
    # This way you only have the piece of the image that you're concerned with.
    collection = ee.ImageCollection(sensor['name']) \
                .filterDate(ee.Date(start_date), ee.Date(end_date)) \
                .filterBounds(roi) \
                ### NOTE: Does this need the lambda??
                .map( lambda x: clipToROI(x, ee.Geometry(roi)) )

    # If there are filters_before, apply them
    if filters_before is not None:
        collection = collection.filter( filters_before )

    # If there are additional functions you want to apply, put them in the
    # "modifiers" list and then they will be applied in turn using the 'map' method
    if modifiers and len(modifiers) > 0:
        for m in modifiers:
            collection = collection.map(m)

    # If there are filters_after, apply them
    if filters_after:
        collection = collection.filter( filters_after )

    # 'sensor' states the specific bands you want to take in the 'bands' value.
    # Return those bands of the image collection.
    # This is done at the end just in case other bands are used in custom (pre-)processing--
    # i.e., in the "modifiers" list
    return collection.select(sensor['bands'])

## process_datasource

In [18]:
def process_datasource(task_queue, source, sensor, export_to, export_dest, feature_list = None):
    # feature_list = ee.FeatureCollection(source['features_src'])
    feature_list = feature_list.sort(source['sort_by']).toList(feature_list.size())
    n_features = feature_list.size().getInfo()

    print("{} features have been loaded".format(n_features))

    task_list = []

    for i in range(1, n_features):
        feature_point = ee.Feature( feature_list.get(i) )

        if source['geometry'] == "point":
            feature_point = feature_point.buffer(source['size']).bounds()

        roi = feature_point.geometry()
        roi = roi.coordinates().getInfo()

        if isinstance(source['name'], str):
            source['name'] = [source['name']]

        if isinstance(sensor['prefix'], str):
            sensor['prefix'] = [sensor['prefix']]

        if 'prefix' in sensor:
            filename_parts = sensor['prefix'] + source['name']
        else:
            filename_parts = source['name']

        filename = "_".join(source['name'] + [str(i)])
        dest_path = "/".join(filename_parts + [filename])

        export_params = {
            'bucket': export_dest,
            'resolution': source['resolution'],
            'filename': filename,
            'dest_path': dest_path
        }

        task_params = {
            'action': export_single_feature,
            'id': "_".join(filename_parts + [str(i)]), # This must be unique per task, to allow to track retries
            'kwargs': {
                'roi': roi,
                'export_params': export_params,
                'sensor': sensor,
                'date_range': {'start_date': source['start_date'], 'end_date': source['end_date']}
            }
        }

        task_queue.add_task(task_params, blocking=True)

In [ ]:
def process_datasource(task_queue, source, sensor, export_to, export_dest, feature_list = None)
### NOTE: We're going to remove the task_queue probably

In [ ]:
# the feature_list is an ee.FeatureCollection
# This sorts the feature_list by the parameter in source['sort_by']
feature_list = feature_list.sort(source['sort_by']).toList(feature_list.size())
# get the number of features in the feature_list
n_features = feature_list.size().getInfo()

In [21]:
fc

In [24]:
fc.toList(fc.size())

In [25]:
help(fc.toList)

Help on method Collection.toList in Collection:

Collection.toList(*args, **kwargs) method of ee.featurecollection.FeatureCollection instance
    Returns the elements of a collection as a list.
    
    Args:
      collection: The input collection to fetch.
      count: The maximum number of elements to fetch.
      offset: The number of elements to discard from the start. If
          set, (offset + count) elements will be fetched and the
          first offset elements will be discarded.



In [26]:
fc.size().getInfo()

3

In [27]:
task_list = []
# This variable is not used so I don't know why it's defined